读取知识库的 Markdown 文档并进行数据处理

In [ ]:
# 代码块1
# 读取 Markdown 数据

r"""
pip install LangChain langchain-community "unstructured[md]"

在安装完成依赖后, 如果报错 Resource xxx not found, import nltk, nltk.download('xxx')
可以将 DataBase 中的 xxx.zip 解压后放置在下面的任一一个目录中

venv\nltk_data'
venv\share\nltk_data'
venv\lib\nltk_data

这里以 Attempted to load taggers/averaged_perceptron_tagger_eng/ 这个错误提示为例
放置完成后的路径如 \venv\share\nltk_data\taggers\averaged_perceptron_tagger_eng(一般是解压得到的文件夹名) 所示
"""

from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader(
    "../DataBase/KnowledgeDB/使用fail2ban配置SSH防爆破.md"
)
md_pages = loader.load()

print(
    f"载入后的变量类型为：{type(md_pages)}，",
    f"该 Markdown 一共包含 {len(md_pages)} 页",
)

md_page = md_pages[0]
print(
    f"每一个元素的类型：{type(md_page)}.",
    f"该文档的描述性数据：{md_page.metadata}",
    # f"查看该文档的内容:\n{md_page.page_content[0:][:2000]}",
    sep="\n------\n",
)

In [ ]:
# 代码块2
# 数据清洗

import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)

# 使用正则表达式或者replace函数处理读取得到的内容
md_page.page_content = md_page.page_content.replace('\n\n', '\n')
print(md_page.page_content)

In [ ]:
# 代码块3
# 文档分隔

''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
text_splitter.split_text(md_page.page_content[0:1000])

split_docs = text_splitter.split_documents(md_pages)
print(f"切分后的文件数量：{len(split_docs)}")

print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")